## Agents
- The core idea of agents is to use a language model to choose a sequence of actions to take.
- In chains, we predefine a sequence of actions. With agents, the language model is used as a reasoning engine to generate a sequence of actions to take and in which order.
- The first thing to do while creating any type of agent is to define which type of tools it should have access to.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', '')

In [3]:
# Run this cell if you want to make your display wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
# Initialize the model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=openai_api_key)

In [5]:
# Open AI Embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

### Example 1 - Retrieval and Web Search Tools
- For this example we will give the agent access to 2 tools.
    1. The retriever for documents created as part of retrieval chain
    2. A search tool. This will search the web for more up to date information.

In [6]:
# Load the data to index
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [9]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

In [11]:
# Create a retriever from the document
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [12]:
# Now setup a tool for the retriever created in above step
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

For the search tool we will use Tavily API - https://tavily.com/#api. This is just to understand how tools work. Later will try to create my own search tool to see how tool creation works.

In [14]:
from langchain_community.tools.tavily_search import TavilySearchResults
load_dotenv()

search = TavilySearchResults()

Now we can create a list of tools we want to search with.

In [17]:
tools = [retriever_tool, search]

In [15]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [18]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_

In [19]:
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now invoke the agent and see how it responds! We can ask it questions about LangSmith

In [20]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith





Skip to main contentü¶úÔ∏èüõ†Ô∏è LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction‚ÄãLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start‚ÄãTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps‚ÄãCheck out the following sections to learn more about LangSmith:User Gui

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform for building production-grade Language Model (LLM) applications. It helps with debugging, testing, evaluating, and monitoring chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs. \n\nYou can use LangSmith for tracing, evaluation, and monitoring your LLM applications. It supports workflows at each stage of the LLM application lifecycle and provides capabilities for tracing and evaluation. Additionally, LangSmith offers a Prompt Hub, a prompt management tool built into the platform.\n\nIf you are interested in using LangSmith for testing, you can leverage its tracing and evaluation capabilities to test and evaluate your LLM applications effectively.'}

In [21]:
# We can ask it about the weather:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://weather.com/weather/hourbyhour/l/USCA0987:1:US', 'content': "recents\nSpecialty Forecasts\nHourly Weather-San Francisco, CA\nBeach Hazard Statement\nSaturday, November 25\n5 pm\nClear\n6 pm\nClear\n7 pm\nClear\n8 pm\nMostly Clear\n9 pm\nPartly Cloudy\n10 pm\nPartly Cloudy\n11 pm\nPartly Cloudy\nSunday, November 26\n12 am\nPartly Cloudy\n1 am\nMostly Cloudy\n2 am\nMostly Cloudy\n3 am\nMostly Cloudy\n4 am\nCloudy\n5 am\nCloudy\n6 am\nMostly Cloudy\n7 am\nMostly Cloudy\n8 am\nMostly Cloudy\n9 am\nMostly Cloudy\n10 am\nPartly Cloudy\n11 am\nPartly Cloudy\n12 pm\nPartly Cloudy\n1 pm\nPartly Cloudy\n2 pm\nPartly Cloudy\n3 pm\nPartly Cloudy\n4 pm\nPartly Cloudy\n5 pm\nPartly Cloudy\n6 pm\nPartly Cloudy\n7 pm\nPartly Cloudy\n8 pm\nPartly Cloudy\n9 pm\nMostly Clear\n10 pm\nMostly Clear\n11 pm\nMostly Clear\nMonday, November 27\n12 am\nClear\n1 am\nMost

{'input': 'what is the weather in SF?',
 'output': "The weather in San Francisco is currently clear with temperatures ranging from 50°F to 62°F. There will be mostly sunny conditions during the day and mostly cloudy at night. If you'd like more detailed information, you can visit [this link](https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA)."}

### Deep Dive
- In the next section we will do a deep dive into agents
- Reference documentation - https://python.langchain.com/docs/modules/agents